In [1]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

os.chdir(r"C:\Users\cgree\Documents\Astra\Space_weather5_22\weakley_all")

In [ ]:
from __future__ import absolute_import, division, print_function

import base64
import imageio
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image
import pyvirtualdisplay

import tensorflow as tf

from tf_agents.agents.dqn import dqn_agent
from tf_agents.drivers import dynamic_step_driver
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.eval import metric_utils
from tf_agents.metrics import tf_metrics
from tf_agents.networks import q_network
from tf_agents.policies import random_tf_policy
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

In [ ]:
from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))
import time
import copy
import numpy as np
import pandas as pd
import chainer
import chainer.functions as F
import chainer.links as L
from plotly import tools
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot, iplot_mpl
init_notebook_mode()
import plotly.express as px

In [2]:
df = pd.read_csv('combined_csv.csv')
df.index = pd.to_datetime(df[['year','month','day' ,'hour','minute','second']])
df = df.drop(columns=["year", "month", "day", "hour", "minute", "second"])

In [3]:
df3 = pd.read_csv(r'C:\Users\cgree\Documents\Astra\Space_weather5_22\2018_sm_reg.csv')
df3 = df3.set_index('Date_UTC')

In [4]:
dfx = df.join(df3, how='inner')#  how='left')   #'inner')

In [ ]:
dfx= dfx.replace([99999.9, 9999.99, 999.99, 999999.0 ,999999.00,-99999.990000, '#REF!'], np.nan)

In [ ]:
del dfx['AE']

In [ ]:
def calculate_derivative(df, new_column_name, column_to_derive, shift_in_date = 1):
    df[new_column_name] = (df[column_to_derive])-(df[column_to_derive].shift(1))/(shift_in_date)
    return df[new_column_name]

In [ ]:
new_column_name = 'FOD' #first order derivative
column_to_derive = 'gic'
calculate_derivative(dfx, new_column_name, column_to_derive, shift_in_date = 1)

In [ ]:
#second order derivative
new_column_name = 'SOD' #second order derivative
column_to_derive = 'FOD'
calculate_derivative(dfx, new_column_name, column_to_derive, shift_in_date = 1)

In [ ]:
data = dfx[['gic', 'FOD', 'SOD']]

In [ ]:
data = new_df[2:] #dropping first two rows with Nan

In [ ]:
class Environment1:
    
    def __init__(self, data, history_t=90):
        self.data = data
        self.history_t = history_t
        self.reset()
        
    def reset(self):
        self.t = 0
        self.done = False
        self.profits = 0
        self.positions = []
        self.position_value = 0
        self.history = [0 for _ in range(self.history_t)]
        return [self.position_value] + self.history # obs
    
    def step(self, act):
        reward = 0
        
        # act = 0: stay, 1: up(left), 2: down(right)
        if act == 1: #up
            self.positions.append(self.data.iloc[self.t, :]['gic'])
        elif act == 2: # down
            if len(self.positions) == 0:
                reward = - 1
            else:
                profits = 0
                for p in self.positions:
                    profits += 1 #(self.data.iloc[self.t, :]['gic'] - p)
                reward += profits
                self.profits += profits
                self.positions = []
        
        # set next time
        self.t += 1
        self.position_value = 0
        for p in self.positions:
            self.position_value += (self.data.iloc[self.t, :]['gic'] - p)
        self.history.pop(0)
        self.history.append(self.data.iloc[self.t, :]['FOD'])
        
        # clipping reward
        if reward > 0:
            reward = 1
        elif reward < 0:
            reward = -1
        
        return [self.position_value] + self.history, reward, self.done # obs, reward, done

In [ ]:
date_split = '2018-03-15 00:00:00'
# train = data[:date_split]
# test = data[date_split:]
train = data['2018-03-01 00:00:00':'2018-03-15 00:00:00']
test = data['2018-03-15 01:00:00':'2018-03-31 23:59:00']
len(train), len(test)

In [ ]:
env = Environment1(train)
print(env.reset())
for _ in range(3):
    pact = np.random.randint(3)
    print(env.step(pact))

In [ ]:
# Double DQN

def train_ddqn(env):

    class Q_Network(chainer.Chain): #Chainer is similar to Open AI its for RL archetecture
    #'''look over documentaion if you forget what this means--https://chainer.org/general/2017/02/22/ChainerRL-Deep-Reinforcement-Learning-Library.html'''
        def __init__(self, input_size, hidden_size, output_size):
            super(Q_Network, self).__init__(
                fc1 = L.Linear(input_size, hidden_size),
                fc2 = L.Linear(hidden_size, hidden_size),
                fc3 = L.Linear(hidden_size, output_size)
            )

        def __call__(self, x):
            h = F.relu(self.fc1(x))
            h = F.relu(self.fc2(h))
            y = self.fc3(h)
            return y

        def reset(self):
            self.zerograds()

    #Then “Agent” can be defined given the model, an optimizer in Chainer, and algorithm-specific 
    #parameters. Agents execute the training of the model through interactions with the environment.
    Q = Q_Network(input_size=env.history_t+1, hidden_size=100, output_size=3)
    Q_ast = copy.deepcopy(Q)
    optimizer = chainer.optimizers.Adam()
    optimizer.setup(Q)

    epoch_num = 50
    step_max = len(env.data)-1
    memory_size = 200
    batch_size = 50
    epsilon = 1.0
    epsilon_decrease = 1e-3 #these are the parameters for the Q funtion.
    epsilon_min = 0.1
    start_reduce_epsilon = 200
    train_freq = 10
    update_q_freq = 20
    gamma = 0.97  # Set the discount factor that discounts future rewards.
    show_log_freq = 5

    memory = []
    total_step = 0
    total_rewards = []
    total_losses = []

    start = time.time()
    for epoch in range(epoch_num):

        pobs = env.reset()
        step = 0
        done = False
        total_reward = 0
        total_loss = 0

        while not done and step < step_max:

            # select act
            pact = np.random.randint(3)
            if np.random.rand() > epsilon:
                pact = Q(np.array(pobs, dtype=np.float32).reshape(1, -1))
                pact = np.argmax(pact.data)

            # act
            obs, reward, done = env.step(pact)

            # add memory
            memory.append((pobs, pact, reward, obs, done))
            if len(memory) > memory_size:
                memory.pop(0)

            # train or update q
            if len(memory) == memory_size:
                if total_step % train_freq == 0:
                    shuffled_memory = np.random.permutation(memory)
                    memory_idx = range(len(shuffled_memory))
                    for i in memory_idx[::batch_size]:
                        batch = np.array(shuffled_memory[i:i+batch_size])
                        b_pobs = np.array(batch[:, 0].tolist(), dtype=np.float32).reshape(batch_size, -1)
                        b_pact = np.array(batch[:, 1].tolist(), dtype=np.int32)
                        b_reward = np.array(batch[:, 2].tolist(), dtype=np.int32)
                        b_obs = np.array(batch[:, 3].tolist(), dtype=np.float32).reshape(batch_size, -1)
                        b_done = np.array(batch[:, 4].tolist(), dtype=np.bool)

                        q = Q(b_pobs)
                        """ <<< DQN -> Double DQN
                        maxq = np.max(Q_ast(b_obs).data, axis=1)
                        === """
                        indices = np.argmax(q.data, axis=1)
                        maxqs = Q_ast(b_obs).data
                        """ >>> """
                        target = copy.deepcopy(q.data)
                        for j in range(batch_size):
                            """ <<< DQN -> Double DQN
                            target[j, b_pact[j]] = b_reward[j]+gamma*maxq[j]*(not b_done[j])
                            === """
                            target[j, b_pact[j]] = b_reward[j]+gamma*maxqs[j, indices[j]]*(not b_done[j])
                            """ >>> """
                        Q.reset()
                        loss = F.mean_squared_error(q, target)
                        total_loss += loss.data
                        loss.backward()
                        optimizer.update()

                if total_step % update_q_freq == 0:
                    Q_ast = copy.deepcopy(Q)

            # epsilon
            if epsilon > epsilon_min and total_step > start_reduce_epsilon:
                epsilon -= epsilon_decrease

            # next step
            total_reward += reward
            pobs = obs
            step += 1
            total_step += 1

        total_rewards.append(total_reward)
        total_losses.append(total_loss)

        if (epoch+1) % show_log_freq == 0:
            log_reward = sum(total_rewards[((epoch+1)-show_log_freq):])/show_log_freq
            log_loss = sum(total_losses[((epoch+1)-show_log_freq):])/show_log_freq
            elapsed_time = time.time()-start
            print('\t'.join(map(str, [epoch+1, epsilon, total_step, log_reward, log_loss, elapsed_time])))
            start = time.time()
            
    return Q, total_losses, total_rewards

In [ ]:
Q, total_losses, total_rewards = train_ddqn(Environment1(train))

In [ ]:
def plot_train_test_by_q(train_env, test_env, Q, algorithm_name):
    
    # train
    pobs = train_env.reset()
    train_acts = []
    train_rewards = []

    for _ in range(len(train_env.data)-1):
        
        pact = Q(np.array(pobs, dtype=np.float32).reshape(1, -1))
        pact = np.argmax(pact.data)
        train_acts.append(pact)
            
        obs, reward, done = train_env.step(pact)
        train_rewards.append(reward)

        pobs = obs
        
    train_profits = train_env.profits
    
    # test
    pobs = test_env.reset()
    test_acts = []
    test_rewards = []

    for _ in range(len(test_env.data)-1):
    
        pact = Q(np.array(pobs, dtype=np.float32).reshape(1, -1))
        pact = np.argmax(pact.data)
        test_acts.append(pact)
            
        obs, reward, done = test_env.step(pact)
        test_rewards.append(reward)

        pobs = obs
        
    test_profits = test_env.profits
    
    # plot
    train_copy = train_env.data.copy()
    test_copy = test_env.data.copy()
    train_copy['act'] = train_acts + [np.nan]
    train_copy['reward'] = train_rewards + [np.nan]
    test_copy['act'] = test_acts + [np.nan]
    test_copy['reward'] = test_rewards + [np.nan]
    train0 = train_copy[train_copy['act'] == 0]
    train1 = train_copy[train_copy['act'] == 1]
    train2 = train_copy[train_copy['act'] == 2]
    test0 = test_copy[test_copy['act'] == 0]
    test1 = test_copy[test_copy['act'] == 1]
    test2 = test_copy[test_copy['act'] == 2]
    act_color0, act_color1, act_color2 = 'gray', 'cyan', 'magenta'

    data = [
        Scatter(x=train0.index, y=train0['gic'],mode='markers',marker_color=act_color0,text=train0['gic']), # hover text goes here,
        Scatter(x=train1.index, y=train0['gic'],mode='markers',marker_color=act_color1,text=train1['gic']),
        Scatter(x=train2.index, y=train0['gic'],mode='markers',marker_color=act_color2,text=train2['gic']),
        Scatter(x=test0.index, y=test0['gic'],mode='markers',marker_color=act_color0,text=train0['gic']),
        Scatter(x=test1.index, y=test1['gic'],mode='markers',marker_color=act_color1,text=train1['gic']),
        Scatter(x=test2.index, y=test2['gic'],mode='markers',marker_color=act_color2,text=train2['gic'])
    ]
#     data = [
#         Candlestick(x=train0.index, open=train0['gic'], high=train0['FOD'], low=train0['SOD'], close=train0['gic'], increasing=dict(line=dict(color=act_color0)), decreasing=dict(line=dict(color=act_color0))),
#         Candlestick(x=train1.index, open=train1['gic'], high=train1['FOD'], low=train1['SOD'], close=train1['gic'], increasing=dict(line=dict(color=act_color1)), decreasing=dict(line=dict(color=act_color1))),
#         Candlestick(x=train2.index, open=train2['gic'], high=train2['FOD'], low=train2['SOD'], close=train2['gic'], increasing=dict(line=dict(color=act_color2)), decreasing=dict(line=dict(color=act_color2))),
#         Candlestick(x=test0.index, open=test0['gic'], high=test0['FOD'], low=test0['SOD'], close=test0['gic'], increasing=dict(line=dict(color=act_color0)), decreasing=dict(line=dict(color=act_color0))),
#         Candlestick(x=test1.index, open=test1['gic'], high=test1['FOD'], low=test1['SOD'], close=test1['gic'], increasing=dict(line=dict(color=act_color1)), decreasing=dict(line=dict(color=act_color1))),
#         Candlestick(x=test2.index, open=test2['gic'], high=test2['FOD'], low=test2['SOD'], close=test2['gic'], increasing=dict(line=dict(color=act_color2)), decreasing=dict(line=dict(color=act_color2)))
#     ]
    title = '{}: train s-reward {}, profits {}, test s-reward {}, profits {}'.format(
        algorithm_name,
        int(sum(train_rewards)),
        int(train_profits),
        int(sum(test_rewards)),
        int(test_profits)
    )
    layout = {
        'title': title,
        'showlegend': False,
         'shapes': [
             {'x0': date_split, 'x1': date_split, 'y0': 0, 'y1': 1, 'xref': 'x', 'yref': 'paper', 'line': {'color': 'rgb(0,0,0)', 'width': 1}}
         ],
        'annotations': [
            {'x': date_split, 'y': 1.0, 'xref': 'x', 'yref': 'paper', 'showarrow': False, 'xanchor': 'left', 'text': ' test data'},
            {'x': date_split, 'y': 1.0, 'xref': 'x', 'yref': 'paper', 'showarrow': False, 'xanchor': 'right', 'text': 'train data '}
        ]
    }
    figure = Figure(data=data, layout=layout)
    iplot(figure)

In [ ]:

plot_train_test_by_q(Environment1(train), Environment1(test), Q, 'DQN')